### Installation

In [ ]:
!pip install datasets

In [ ]:
!pip install googletrans

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.7 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15718 sha256=36c85b7ee7c0059f725c38d66ee63733370f6b7f786c72f912cc26692d56f69b
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
   

### Library import

In [ ]:
import pandas as pd
from datasets import load_dataset
import re
import nltk
nltk.download('averaged_perceptron_tagger')
from collections import Counter
from googletrans import Translator
from nltk import word_tokenize, pos_tag
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, classification_report

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### Dataset Loading

In [ ]:
dataset = load_dataset("coastalcph/tydi_xor_rc", split=['train', 'validation'])

train = pd.DataFrame(dataset[0])
validation = pd.DataFrame(dataset[1])

df = pd.concat([train, validation], ignore_index=True)
df.to_excel("tydi_xor_re.xlsx", index=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.85k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/6.87M [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15326 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3028 [00:00<?, ? examples/s]

In [ ]:
df.head()

,question,context,lang,answerable,answer_start,answer,answer_inlang
0,উইকিলিকস কত সালে সর্বপ্রথম ইন্টারনেটে প্রথম তথ...,WikiLeaks () is an international non-profit or...,bn,True,182,2006,None
1,দ্বিতীয় বিশ্বযুদ্ধে কোন দেশ পরাজিত হয় ?,The war in Europe concluded with an invasion o...,bn,True,48,Germany,None
2,মার্কিন যুক্তরাষ্ট্রের সংবিধান অনুযায়ী মার্কিন...,Same-sex marriage in the United States expande...,bn,False,-1,no,None
3,আরব-ইসরায়েলি যুদ্ধে আরবের মোট কয়জন সৈন্যের মৃ...,The exact number of Arab casualties is unknown...,bn,True,39,unknown,None
4,বিশ্বে প্রথম পুঁজিবাদী সমাজ কবে গড়ে ওঠে ?,"As Thomas Hall (2000) notes, ""The Sung Empire ...",bn,True,1219,17th century,None


### Finnish Language

loading finnish language

In [ ]:
finnish_train = train[train['lang'] == 'fi']
finnish_validation = validation[validation['lang'] == 'fi']


In [ ]:
finnish_train.head()
finnish_validation.head()

,question,context,lang,answerable,answer_start,answer,answer_inlang
311,Missä maassa Jack Churchill syntyi?,"Churchill was born at Colombo, British Ceylon ...",fi,True,22,"Colombo, British Ceylon",None
312,Mikä on yleisin uskonto maailmassa?,The five largest religious groups by world pop...,fi,True,130,Christianity,None
313,Kuka oli Glee sarjan pääosassa?,Rachel Barbra Berry (Lea Michele) is the lead ...,fi,True,0,Rachel Barbra Berry,None
314,Milloin Killzone-sarjan peli julkaistiin ensim...,Killzone is a series of first-person shooter a...,fi,True,404,November 2004,None
315,Milloin Pennsylvania liitty USA?,The state is one of the 13 original founding s...,fi,True,404,"December 12, 1787",None


Number of finnish samples

In [ ]:
num_train_samples = len(finnish_train)
num_validation_samples = len(finnish_validation)

print(f"Number of Finnish samples in the training set: {num_train_samples}")
print(f"Number of Finnish samples in the validation set: {num_validation_samples}")


Number of Finnish samples in the training set: 2126
Number of Finnish samples in the validation set: 528


5 Most common word in the question of trainning data

In [ ]:
finnish_questions = ' '.join(finnish_train['question'].tolist())

question_words = re.findall(r'\b\w+\b', finnish_questions.lower())

word_counts = Counter(question_words)

most_common_words = word_counts.most_common(5)

for word, count in most_common_words:
    print(f"Word: '{word}', Occurrences: {count}")


Word: 'on', Occurrences: 642
Word: 'mikä', Occurrences: 328
Word: 'milloin', Occurrences: 287
Word: 'vuonna', Occurrences: 227
Word: 'kuka', Occurrences: 215


In [ ]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fi-en")

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/832k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/790k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


parts of speech of translated finnish words

In [ ]:
translated_words = [translator(word)[0]['translation_text'] for word, _ in most_common_words]

pos_tags = pos_tag(translated_words)

for word, pos in pos_tags:
    print(f"Word: '{word}', POS Tag: {pos}")


Word: 'is', POS Tag: VBZ
Word: 'what', POS Tag: WP
Word: 'when', POS Tag: WRB
Word: 'year', POS Tag: NN
Word: 'who', POS Tag: WP


## Model

In [ ]:
train_fi = train[train['lang'] == 'fi']
validation_fi = validation[validation['lang'] == 'fi']


In [ ]:
def translate_to_english(question):
    return translator(question)[0]['translation_text']

train_fi['translated_question'] = train_fi['question'].apply(translate_to_english)

<ipython-input-28-1f4304e96d7d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_fi['translated_question'] = train_fi['question'].apply(translate_to_english)


In [ ]:
train_fi.head()

,question,context,lang,answerable,answer_start,answer,answer_inlang,predicted,predicted_sim
9137,Mitkä olivat Rooman alkuvaiheet?,"In historiography, ancient Rome is Roman civil...",fi,True,0,"In historiography, ancient Rome is Roman civil...",None,False,False
9138,Kuka oli toisen maailmansodan jälkeisen sosial...,Rákosi had difficulty managing the economy and...,fi,True,187,Mátyás Rákosi,None,False,False
9139,Mikä oli roomalaisten antama nimi nykyisen Unk...,Hungary in its modern (post-1946) borders roug...,fi,True,286,Pannonia,None,False,False
9140,Kuinka monta ihmistä menehtyi Suezin kriisin a...,"On 25 January 1952, British forces attempted t...",fi,True,131,deaths of 41 Egyptians,None,False,False
9141,Millä vuosikymmenellä Yhdysvaltojen varhaishis...,The history of the United States began with th...,fi,True,87,"15,000 BC",None,False,False


Similarity base class

In [ ]:
def rule_based_classifier_similarity(context, question):
    context_words = set(context.lower().split())
    question_words = set(question.lower().split())

    overlap = len(context_words.intersection(question_words))

    if overlap > 3:
        return True
    return False


In [ ]:
train_fi['predicted_sim'] = train_fi.apply(lambda row: rule_based_classifier_similarity(row['context'], row['question']), axis=1)

validation_fi['predicted_sim'] = validation_fi.apply(lambda row: rule_based_classifier_similarity(row['context'], row['question']), axis=1)


<ipython-input-21-aead9b601afc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_fi['predicted_sim'] = train_fi.apply(lambda row: rule_based_classifier_similarity(row['context'], row['question']), axis=1)
<ipython-input-21-aead9b601afc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_fi['predicted_sim'] = validation_fi.apply(lambda row: rule_based_classifier_similarity(row['context'], row['question']), axis=1)


In [ ]:
# Evaluate accuracy on training data
train_accuracy = accuracy_score(train_fi['answerable'], train_fi['predicted_sim'])
print(f"Training Accuracy: {train_accuracy}")

# Evaluate accuracy on validation data
validation_accuracy = accuracy_score(validation_fi['answerable'], validation_fi['predicted_sim'])
print(f"Validation Accuracy: {validation_accuracy}")


Training Accuracy: 0.12935089369708372
Validation Accuracy: 0.2821969696969697


Cosine vector

In [ ]:
def rule_based_classifier_cosine(context, question):
    texts = [context, question]
    vectorizer = TfidfVectorizer().fit_transform(texts)
    vectors = vectorizer.toarray()

    similarity_score = cosine_similarity(vectors[0].reshape(1, -1), vectors[1].reshape(1, -1))[0][0]

    if similarity_score > 0.3:
        return True
    return False


In [ ]:
train_fi['predicted_cos'] = train_fi.apply(lambda row: rule_based_classifier_cosine(row['context'], row['question']), axis=1)

validation_fi['predicted_cos'] = validation_fi.apply(lambda row: rule_based_classifier_cosine(row['context'], row['question']), axis=1)


<ipython-input-24-b0fd20c87a42>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_fi['predicted_cos'] = train_fi.apply(lambda row: rule_based_classifier_cosine(row['context'], row['question']), axis=1)
<ipython-input-24-b0fd20c87a42>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_fi['predicted_cos'] = validation_fi.apply(lambda row: rule_based_classifier_cosine(row['context'], row['question']), axis=1)


In [ ]:
train_accuracy = accuracy_score(train_fi['answerable'], train_fi['predicted_cos'])
print(f"Training Accuracy: {train_accuracy}")

train_report = classification_report(train_fi['answerable'], train_fi['predicted_cos'])
print(f"Training Classification Report:\n{train_report}")

validation_accuracy = accuracy_score(validation_fi['answerable'], validation_fi['predicted_cos'])
print(f"Validation Accuracy: {validation_accuracy}")

validation_report = classification_report(validation_fi['answerable'], validation_fi['predicted_cos'])
print(f"Validation Classification Report:\n{validation_report}")


Training Accuracy: 0.12464722483537159
Training Classification Report:
              precision    recall  f1-score   support

       False       0.12      1.00      0.22       263
        True       1.00      0.00      0.00      1863

    accuracy                           0.12      2126
   macro avg       0.56      0.50      0.11      2126
weighted avg       0.89      0.12      0.03      2126

Validation Accuracy: 0.2803030303030303
Validation Classification Report:
              precision    recall  f1-score   support

       False       0.28      1.00      0.44       148
        True       0.00      0.00      0.00       380

    accuracy                           0.28       528
   macro avg       0.14      0.50      0.22       528
weighted avg       0.08      0.28      0.12       528



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
train_fi.head()

,question,context,lang,answerable,answer_start,answer,answer_inlang,predicted,predicted_sim,predicted_cos
9137,Mitkä olivat Rooman alkuvaiheet?,"In historiography, ancient Rome is Roman civil...",fi,True,0,"In historiography, ancient Rome is Roman civil...",None,False,False,False
9138,Kuka oli toisen maailmansodan jälkeisen sosial...,Rákosi had difficulty managing the economy and...,fi,True,187,Mátyás Rákosi,None,False,False,False
9139,Mikä oli roomalaisten antama nimi nykyisen Unk...,Hungary in its modern (post-1946) borders roug...,fi,True,286,Pannonia,None,False,False,False
9140,Kuinka monta ihmistä menehtyi Suezin kriisin a...,"On 25 January 1952, British forces attempted t...",fi,True,131,deaths of 41 Egyptians,None,False,False,False
9141,Millä vuosikymmenellä Yhdysvaltojen varhaishis...,The history of the United States began with th...,fi,True,87,"15,000 BC",None,False,False,False
